# Import tools

In [159]:
import numpy as np
import pandas as pd
from decision_tree import DecisionTreeClassifier
from sklearn.datasets import load_iris

# Load the data

In [160]:
# The amount of samples to use from the dataset
n_samples = 1000

# Times (unloaded)
# 1.000 : 4 seconds (30%)
# 1.500 : 10 seconds (34%)
# 2.000 : 13 seconds (39%)
# 3.000 : 30 seconds (40%)
# 10.000 : 6 minutes (38%)

df_train = pd.read_csv('../bpi_2012_train.csv')
df_train = df_train[['eventID ', 'case REG_DATE', 'event lifecycle:transition', 'event time:timestamp', 'event concept:name']].set_index('eventID ')[0:n_samples]

#df_test = pd.read_csv('../bpi_2012_test.csv')
#df_test = df_test[['eventID ', 'case concept:name', 'case REG_DATE', 'case AMOUNT_REQ', 'event concept:name', 'event lifecycle:transition', 'event time:timestamp']].set_index('eventID ')[0:100]
#df_test.info()


iris = load_iris()

def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

df_iris = sklearn_to_df(load_iris())
#df_iris.head()
df_train.head()

,case REG_DATE,event lifecycle:transition,event time:timestamp,event concept:name
eventID,,,,
0,2011-10-01T01:38:44.546+03:00,COMPLETE,2011-01-10 01:38:44.546,A_SUBMITTED
1,2011-10-01T01:38:44.546+03:00,COMPLETE,2011-01-10 01:38:44.880,A_PARTLYSUBMITTED
2,2011-10-01T01:38:44.546+03:00,COMPLETE,2011-01-10 01:39:37.906,A_PREACCEPTED
3,2011-10-01T01:38:44.546+03:00,SCHEDULE,2011-01-10 01:39:38.875,W_Completeren aanvraag
4294967296,2011-10-01T09:08:58.256+03:00,COMPLETE,2011-01-10 09:08:58.256,A_SUBMITTED


# Splitting the training and testing data

As far as I'm aware, this step has already been performed by Nik. For now however, I'll attempt to reproduce the same split as programmed by my source of inspiration for the Decision Tree Classifier model to check whether it works.

In [161]:
from sklearn.model_selection import train_test_split

X = df_train.iloc[:, :-1].values # The original values without the target variable
Y = df_train.iloc[:, -1].values.reshape(-1, 1) # Turn the target variable into a single-column array of arrays

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

pd.DataFrame(X).head(20)
pd.DataFrame(Y).head(10)

,0
0,A_SUBMITTED
1,A_PARTLYSUBMITTED
2,A_PREACCEPTED
3,W_Completeren aanvraag
4,A_SUBMITTED
5,A_PARTLYSUBMITTED
6,A_PREACCEPTED
7,W_Completeren aanvraag
8,A_SUBMITTED
9,A_PARTLYSUBMITTED


# Fitting the model

In [162]:
classifier = DecisionTreeClassifier(min_samples_split=10, max_depth=10)
classifier.fit(X_train, Y_train)
classifier.print_tree()

X_1 <= COMPLETE ? 0.030055233261584546
 left:X_2 <= 2011-01-11 08:22:58.171 ? 0.01926292396164675
  left:X_0 <= 2011-10-01T17:12:02.338+03:00 ? 0.029356684703979785
    left:X_2 <= 2011-01-10 11:38:20.967 ? 0.02677510348821066
        left:X_2 <= 2011-01-10 10:59:10.789 ? 0.04611147899389312
                left:X_0 <= 2011-10-01T10:45:25.806+03:00 ? 0.027823691460055144
                                left:X_2 <= 2011-01-10 01:38:44.880 ? 0.05269841269841269
                                                                left:A_PARTLYSUBMITTED
                                                                right:X_0 <= 2011-10-01T01:38:44.546+03:00 ? 0.05416012558869698
                                                                                                                                left:A_PREACCEPTED
                                                                                                                                right:X_2 <= 2011-01-10 10:45:25.806 ? 0.0680

# Prediction and analysis

In [163]:
from sklearn.metrics import accuracy_score

Y_pred = classifier.predict(X_test)
accuracy_score(Y_test, Y_pred)

0.295